In [29]:
# ignore warnings
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [30]:
# imports
from pathlib import Path
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import hvplot.pandas
import hvplot.dask


### cleaning data

In [35]:
# name csv path
ibonds_path = Path("US_Treasury_Series_I_Bond_Historical_Rates.csv")

# read csv
ibonds_df =pd.read_csv(ibonds_path)

ibonds_df.head()

,Date the rate was set,Fixed rate for bonds issued in the six months after that date,Inflation rate for all I bonds issued for six months (starting in that bond's next interest start month)
0,November 1 2022,0.4,3.24
1,May 1 2022,0.0,4.81
2,November 1 2021,0.0,3.56
3,May 1 2021,0.0,1.77
4,November 1 2020,0.0,0.84


In [36]:
# rename columns
ibonds_df.rename(columns={"Date the rate was set":"date", "Fixed rate for bonds issued in the six months after that date":"fixed rate", "Inflation rate for all I bonds issued for six months (starting in that bond's next interest start month)":"six-month inflation rate"}, inplace=True)

# drop nulls
ibonds_df.dropna(axis=0, inplace=True)

# correct error in start dates
ibonds_df["date"][41] = "May 1 2002"
ibonds_df["date"][42] = "November 1 2001"

# add missing data
# source: https://www.treasurydirect.gov/savings-bonds/i-bonds/i-bonds-interest-rates/
new_rows = {"date":["November 1 2023", "May 1 2023"],
    "fixed rate":["1.3", ".90"],
    "six-month inflation rate":["1.97", "1.69"]
}

# create new df
new_rows_df = pd.DataFrame(new_rows)

# concat dataframes
ibonds_combined_df = pd.concat([new_rows_df, ibonds_df])
ibonds_combined_df

# set date to datetime
ibonds_combined_df["date"] = pd.to_datetime(ibonds_combined_df["date"], format="%B %d %Y")

# reset index
ibonds_combined_df.reset_index(inplace=True)

# order ascending
ibonds_combined_df.sort_index(inplace=True, ascending=False)

# set date back as string
ibonds_combined_df["date"] = pd.to_datetime(ibonds_combined_df["date"]).dt.strftime("%m-%Y")

# set index
ibonds_combined_df.set_index("date", inplace=True)

# drop "index"
ibonds_combined_df.drop(columns=["index"], inplace=True)

# set dtypes to correct the manually entered data
ibonds_combined_df["fixed rate"] = ibonds_combined_df["fixed rate"].astype(float)
ibonds_combined_df["six-month inflation rate"] = ibonds_combined_df["six-month inflation rate"].astype(float)

# export cleaned df to csv for other uses
ibonds_combined_df.to_csv("ibonds_cleaned_data.csv")

ibonds_combined_df.head()

print(ibonds_combined_df)

         fixed rate  six-month inflation rate
date                                         
09-1998         3.4                      0.62
11-1998         3.3                      0.86
05-1999         3.3                      0.86
11-1999         3.4                      1.76
05-2000         3.6                      1.91
11-2000         3.4                      1.52
05-2001         3.0                      1.44
11-2001         2.0                      1.19
05-2002         2.0                      0.28
11-2002         1.6                      1.23
05-2003         1.1                      1.77
11-2003         1.1                      0.54
05-2004         1.0                      1.19
11-2004         1.0                      1.33
05-2005         1.2                      1.79
11-2005         1.0                      2.85
05-2006         1.4                      0.50
11-2006         1.4                      1.55
05-2007         1.3                      1.21
11-2007         1.2               

### graphs

calculations

In [37]:
# annualize variable interest rate
ibonds_combined_df["annualized six-month variable interest rate"] = np.maximum((ibonds_combined_df["six-month inflation rate"] * 2), 0)

# combined rate
ibonds_combined_df["aggregate annualized interest rate"] = ibonds_combined_df["annualized six-month variable interest rate"] + ibonds_combined_df["fixed rate"]

ibonds_combined_df.head()

,fixed rate,six-month inflation rate,annualized six-month variable interest rate,aggregate annualized interest rate
date,,,,
09-1998,3.4,0.62,1.24,4.64
11-1998,3.3,0.86,1.72,5.02
05-1999,3.3,0.86,1.72,5.02
11-1999,3.4,1.76,3.52,6.92
05-2000,3.6,1.91,3.82,7.42


stacked bar chart

In [41]:
ibonds_bar = ibonds_combined_df.hvplot.bar(
    x="date",
    y=["fixed rate", "annualized six-month variable interest rate"],
    # set ylim to zero because interest cannot be negative on ibonds
    ylim=(0,12),
    stacked=True,
    color=["orange", "red"],
    title="I bonds Aggregate Annualized Interest Rate",
    ylabel="Interest Rate %",
    rot=70,
    xlabel="",
    width=1000
    )

hvplot.save(ibonds_bar, "ibonds_bar.html")

ibonds_bar



:Bars   [date,Variable]   (value)